In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from IPython.display import clear_output
import re

In [3]:
wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/matches')

SessionNotCreatedException: Message: session not created
from chrome not reachable
  (Session info: chrome=75.0.3770.100)


### 1. Definujemy funkcje potrzebne do pobrania danych

In [76]:

def next_page():
    element = wd.find_elements_by_class_name('pagination-next')
    element[0].click()
    
def if_next_page():
    element = wd.find_elements_by_class_name('pagination-next')
    next_page_exists=True
    if element[0].get_attribute('href') == 'https://www.hltv.org/stats/matches?offset=0':
        next_page_exists=False
    return next_page_exists
    
    
#     return element[0].get_attribute('href')
    
def previous_page():
    element = wd.find_elements_by_class_name('pagination-prev')
    element[0].click()


def go_back_page():
    wd.back()

    
###dane_tabel    
def get_info_from_main_tabels():
    results_table = []
    tables = wd.find_elements_by_class_name('stats-table') 
    
    for table in tables:
        rows = table.find_elements_by_tag_name('tr')
        columns_names=get_columns_names(table)
    
    for row in rows:
        columns = row.find_elements_by_css_selector("td")
        temp = []
        for column in columns:
            temp.append(column.text)
        temp.append(get_link_summary(row))
        results_table.append(temp)
        df_out = pd.DataFrame(results_table)
        
        
    return df_out, columns_names



###nazwy kolumn
def get_columns_names(table):
    columns_names = []
    table_head = table.find_elements_by_tag_name('th')
    for head in table_head:
        columns_names.append(head.text)
    return columns_names
    
    
###pobierz link z glownej strony   
def get_link_summary(row):
    columns_link = row.find_elements_by_class_name('date-col')
    link=''
    for column in columns_link:
        if column.find_elements_by_css_selector('a'):
            temp2 = column.find_elements_by_css_selector('a')[0]
            link = temp2.get_attribute('href')
    return(link)
    
    
def in_database(df, last_element):
    m =  df.isin([last_element]).any()
    m = m.any()
    return m
 
    
## nowe dane do tabeli glownej

def get_new_data_to_main_table():
    df=pd.DataFrame()
    df_new, _ =  get_info_from_main_tabels()
    df = pd.concat([df, df_new])
    last_value=df.iloc[-1][5]

    while ~in_database(df_main_table, last_value):
        next_page()
        df_new, _ =  get_info_from_main_tabels()
        df = pd.concat([df, df_new])
        last_value = df.iloc[-1][5]
    df = process_main_table(df)
    df = df[~df['link'].isin(df_main_table['link'])]
    return df
    
    

    
    
# funkcje do czyszczenia tabeli glownej


#oczysczanie glownej tabeli
def process_main_table(df):
    df = df.dropna()
    columns_df_main = ['date', 'team1', 'team2', 'map', 'event', 'link']
    df.columns = columns_df_main
    df.reset_index(drop=True, inplace=True)
    df['score_team1'] = df.apply(lambda x: get_score_from_main_table(x['team1']), axis=1)
    df['team1'] = df.apply(lambda x: get_name_from_main_table(x['team1']), axis=1)
    df['score_team2'] = df.apply(lambda x: get_score_from_main_table(x['team2']), axis=1)
    df['team2'] = df.apply(lambda x: get_name_from_main_table(x['team2']), axis=1)
    df['date'] = pd.to_datetime(df['date'], format = '%d/%m/%y')
    return df

# wyczysc z nazwy wynik
def get_name_from_main_table(row_text):
    m=re.search('\((.*)\)' , row_text)
    if(m):
        row_text = row_text.replace(" "+m[0],'')
    return row_text    



# wez z nazwy wynik
def get_score_from_main_table(row_text):
    m=re.search('\((.*)\)' , row_text)
    if(m):
        score = m[1]
    else:
        score = np.nan
    return score

    
## FUNKCJE DO TABEL ZE SZCZEGOLAMI

def get_info_from_stats():
    list_info = {}
    list1 = wd.find_element_by_class_name('stats-match-maps') 
    list2 = list1.find_elements_by_class_name('stats-match-map')
    if list2:
        list_info['score_total'] = list2[0].text.split('\n')[0]
        list_info['match_type'] = list2[0].text.split('\n')[1]
        for idx, li in enumerate(list2):
            if(li.get_attribute("class") == 'col stats-match-map standard-box a-reset'):
                list_info['map_number'] = idx
    else:
        list_info['score_total'] = np.nan
        list_info['match_type'] = 'Best of 1'
        list_info['map_number'] = 1    
    return(list_info)

def get_match_info():
    dict_match_info={}
    match_info = wd.find_element_by_class_name('match-info-box-con')
    match_info = match_info.find_element_by_class_name('match-info-box')
    list_match_info = match_info.text.split("\n")
    dict_match_info['event'] = list_match_info[0]
    dict_match_info['date_time'] = list_match_info[1]
    dict_match_info['map_name'] = list_match_info[4]
    dict_match_info['score_team1'] = list_match_info[6]
    dict_match_info['score_team2'] = list_match_info[8]
    dict_match_info['team1'] = list_match_info[5]
    dict_match_info['team2'] = list_match_info[7]
    return(dict_match_info)

def get_match_info_details():
    dict_match_info_more = {}
    match_info = wd.find_element_by_class_name('match-info-box-con')
    match_info = match_info.find_elements_by_class_name('match-info-row')
    list_scores = match_info[0].find_elements_by_css_selector('span')
    dict_match_info_more['team1side'] = list_scores[2].get_attribute("class")
    dict_match_info_more['team1_score_1half']= list_scores[2].text
    dict_match_info_more['team2_score_1half']= list_scores[3].text
    dict_match_info_more['team1_score_2half']= list_scores[4].text
    dict_match_info_more['team2_score_2half']= list_scores[5].text
    dict_match_info_more['team1_rating'] = match_info[1].text.split("\n")[0].split(" ")[0]
    dict_match_info_more['team2_rating'] = match_info[1].text.split("\n")[0].split(" ")[2]
    dict_match_info_more['team1_first_kills'] = match_info[2].text.split("\n")[0].split(" ")[0]
    dict_match_info_more['team2_first_kills'] = match_info[2].text.split("\n")[0].split(" ")[2]
    dict_match_info_more['team1_clutches_won'] = match_info[3].text.split("\n")[0].split(" ")[0]
    dict_match_info_more['team2_clutches_won'] = match_info[3].text.split("\n")[0].split(" ")[2]
    return(dict_match_info_more)

def get_info_players():
    dict_players={}
    tables =wd.find_elements_by_class_name('stats-table') 
    for index_table,table in enumerate(tables):
        rows = table.find_elements_by_tag_name('tr')
        for index, row in enumerate(rows):
            temp = []
            columns = row.find_elements_by_css_selector("td")
            for column in columns:
                temp.append(column.text)
            if temp:
                dict_players.update(players_dict_create(temp, index, index_table+1))
    return (dict_players)

def players_dict_create(row, row_number, team_number):
    dict_to_return={}
    list_of_keys=['player_name','p_kills','p_asists','p_death','p_kast','p_ADR','p_rating']
    player_number = row_number
    dict_to_return[f'team{team_number}_{list_of_keys[0]}_{player_number}']=row[0]
    dict_to_return[f'team{team_number}_{list_of_keys[1]}_{player_number}']=row[1]
    dict_to_return[f'team{team_number}_{list_of_keys[2]}_{player_number}']=row[2]
    dict_to_return[f'team{team_number}_{list_of_keys[3]}_{player_number}']=row[3]
    dict_to_return[f'team{team_number}_{list_of_keys[4]}_{player_number}']=row[4]
    dict_to_return[f'team{team_number}_{list_of_keys[5]}_{player_number}']=row[6]
    dict_to_return[f'team{team_number}_{list_of_keys[6]}_{player_number}']=row[8]
    return dict_to_return
        



    
    



### 2.Pobieramy dane z całosci - tylko gdy musimy odtworzyc cala baze

#### 2.1 pobieramy dane z glownej strony

In [56]:
wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/matches')

In [52]:

#pobieramy dane z glownej
df_all=pd.DataFrame()
while if_next_page():
    df, columns = get_info_from_main_tabels()
    next_page()
    df_all = pd.concat([df_all, df])
    print([wd.find_element_by_class_name('pagination-data').text])
    clear_output(wait=True)

TypeError: 'NoneType' object is not subscriptable

In [54]:
#oczyszczamy dane
df_main = process_main_table(df_all)
df_main.to_pickle('main_table_all_data.pkl')





C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

,date,team1,team2,map,event,link,score_team1,score_team2
0,2019-07-10,DreamEaters,Vega Squadron,Overpass,OGA Counter PIT by AMD and Sapphire Season 5,https://www.hltv.org/stats/matches/mapstatsid/...,7,16
1,2019-07-10,Vega Squadron,DreamEaters,Dust2,OGA Counter PIT by AMD and Sapphire Season 5,https://www.hltv.org/stats/matches/mapstatsid/...,7,16
2,2019-07-10,eNergy,Goliath,Vertigo,ESEA Open Season 31 South Africa,https://www.hltv.org/stats/matches/mapstatsid/...,25,28
3,2019-07-10,PACT,Chaos,Train,OGA Counter PIT by AMD and Sapphire Season 5,https://www.hltv.org/stats/matches/mapstatsid/...,12,16
4,2019-07-10,PACT,Chaos,Overpass,OGA Counter PIT by AMD and Sapphire Season 5,https://www.hltv.org/stats/matches/mapstatsid/...,7,16
5,2019-07-10,HAVU,Ancient,Nuke,GameAgents League Season 3,https://www.hltv.org/stats/matches/mapstatsid/...,16,11
6,2019-07-10,HAVU,Ancient,Dust2,GameAgents League Season 3,https://www.hltv.org/stats/matches/mapstatsid/...,16,4
7,2019-07-10,NoChance,Aristocracy,Train,UCC Summer Smash,https://www.hltv.org/stats/matches/mapstatsid/...,9,16
8,2019-07-10,Aristocracy,NoChance,Nuke,UCC Summer Smash,https://www.hltv.org/stats/matches/mapstatsid/...,16,6
9,2019-07-10,Aristocracy,NoChance,Dust2,UCC Summer Smash,https://www.hltv.org/stats/matches/mapstatsid/...,12,16



#### 2.2 pobieramy dane szczegolowe


In [57]:
df = pd.DataFrame()
number_of_rows = len(df_main.index)

for index, row in df_main.iterrows():
    try:
        dict_temp={}
        link = row['link']
        wd.get(link)
        dict_temp = get_info_from_stats()
        dict_temp.update(get_match_info())
        dict_temp.update(get_match_info_details())
        dict_temp.update(get_info_players())
        dict_temp.update({'link' : link})
        df = df.append(dict_temp,ignore_index=True)
        print(f'{index}/{number_of_rows}')
        clear_output(wait=True)
    except NoSuchElementException:
        pass
    if (index%1000) == 0 :
        df.to_pickle('details_all.pkl')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=75.0.3770.100)


#### 2.3 laczymy wczesniej pobrane bazy

In [58]:
df_main_table = pd.read_pickle('main_table.pkl')
df_details = pd.read_pickle('details.pkl')

In [59]:
df_data = pd.merge(df_main_table, df_details, on=['link'], how='left', suffixes=['main', 'details'] )

In [62]:
df_data.to_pickle('df_data.pkl')

### 3.Pobieramy nowe dane 

#### 3.1 pobieramy z glownej tabeli

In [64]:
wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/matches')
df_main_table = pd.read_pickle('main_table.pkl')

df=pd.DataFrame()
df_new, _ =  get_info_from_main_tabels()
df = pd.concat([df, df_new])
last_value=df.iloc[-1][5]

while ~in_database(df_main_table, last_value):
    next_page()
    df_new, _ =  get_info_from_main_tabels()
    df = pd.concat([df, df_new])
    last_value = df.iloc[-1][5]

In [68]:
df_main = process_main_table(df)

C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

In [69]:
#wycinamy te ktore juz sa w bazie
df_main = df_main[~df_main['link'].isin(df_main_table['link'])]

In [70]:
df_main.to_pickle('main_table_new_data.pkl')

#### 3.2 pobieramy dane szczegolowe

In [71]:
df = pd.DataFrame()
number_of_rows = len(df_main.index)

for index, row in df_main.iterrows():
    try:
        dict_temp={}
        link = row['link']
        wd.get(link)
        dict_temp = get_info_from_stats()
        dict_temp.update(get_match_info())
        dict_temp.update(get_match_info_details())
        dict_temp.update(get_info_players())
        dict_temp.update({'link' : link})
        df = df.append(dict_temp,ignore_index=True)
        print(f'{index}/{number_of_rows}')
        clear_output(wait=True)
    except NoSuchElementException:
        pass
    if (index%1000) == 0 :
        df.to_pickle('details_new.pkl')

853/854


In [72]:
df.to_pickle('details_new.pkl')

#### 3.3 laczymy wczesniej pobrane bazy


In [73]:
df_main_table = pd.read_pickle('main_table_new_data.pkl')
df_details = pd.read_pickle('details_new.pkl')

In [74]:
df_data = pd.merge(df_main_table, df_details, on=['link'], how='left', suffixes=['main', 'details'] )

In [75]:
df_data.to_pickle('df_data_new.pkl')

In [69]:
df_data = pd.read_pickle('df_data.pkl')
df_data_new = pd.read_pickle('df_data_new.pkl')

In [72]:
df_data=pd.concat([df_data, df_data_new])

In [73]:
df_data.to_pickle('df_data_connected.pkl')

In [1]:
#df_data_connected zostanie wykorzysatne do wygenerowania modeli

# 4. Pobieranie Event

In [2]:
wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/events')

In [73]:
def get_data_events():
    df=pd.DataFrame()
    df_new =  get_info_from_main_tabels_event()
    df = pd.concat([df, df_new])
    while if_next_page_events():
        next_page()
        df_new =  get_info_from_main_tabels_event()
        df = pd.concat([df, df_new])
    return df




def if_next_page_events():
    element = wd.find_elements_by_class_name('pagination-next')
    next_page_exists=True
    if element[0].get_attribute('href') == None:
        next_page_exists=False
    return next_page_exists




def get_info_from_main_tabels_event():
    results_table = []
    tables = wd.find_elements_by_class_name('stats-table') 
    
    for table in tables:
        rows = table.find_elements_by_tag_name('tr')
    
    for row in rows:
        columns = row.find_elements_by_css_selector("td")
        temp = []
        for column in columns:
            temp.append(column.text)
        results_table.append(temp)
        df_out = pd.DataFrame(results_table)
             
    return df_out



In [90]:
type_of_event = ['Lan',"Majors", 'BigEvents']
df_events = pd.DataFrame() 

wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/events')
    
for event_type in type_of_event:
    wd.get(f'https://www.hltv.org/stats/events?matchType={event_type}')
    df_temp = get_data_events()
    df_temp['type_of_event'] = event_type
    df_events = pd.concat([df_events,df_temp])


In [104]:
df_events.to_pickle('events.pkl')